### Buffered Leave one Group Out Cross Validation

This Jupyter notebook runs buffered leave-one-group-out cross-validation stratified by mean annual surface air temperature and mean annual precipitation67. This tests the model’s ability to predict CWMs for temperature and precipitation values that have been removed from its training set. For example, if a test group has a mean temperature of 10°C and a 2°C buffer is applied, then all training data with temperatures between 9°C and 11°C are excluded from the model training set. 

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
# Function to rename CHELSA_bio columns consistently
def rename_chelsa_columns(df):
    def clean_chelsa_name(col):
        if col.startswith('CHELSA_bio'):
            parts = col.split('_')
            if len(parts) > 1 and parts[1].startswith('bio'):
                bio_number = parts[1][3:]  # Extract number part from 'bio<number>'
                return f'CHELSA_bio{bio_number}'
        return col

    # Apply the renaming function to all columns
    df.columns = [clean_chelsa_name(col) for col in df.columns]
    return df

# Function to get a consistent numerical order for CHELSA_bio columns
def get_consistent_chelsa_order(*dfs):
    # Collect all CHELSA_bio columns from all DataFrames
    all_chelsa_columns = set()
    for df in dfs:
        chelsa_columns = [col for col in df.columns if col.startswith('CHELSA_bio')]
        all_chelsa_columns.update(chelsa_columns)
    
    # Sort the collected columns numerically
    sorted_chelsa_columns = sorted(all_chelsa_columns, key=lambda x: int(x.replace('CHELSA_bio', '')))
    return sorted_chelsa_columns

# Function to reorder columns based on a given order
def reorder_columns(df, desired_order):
    chelsa_columns = [col for col in df.columns if col.startswith('CHELSA_bio')]
    non_chelsa_columns = [col for col in df.columns if not col.startswith('CHELSA_bio')]
    
    # Ensure that only the columns that exist in the DataFrame are reordered
    chelsa_columns = [col for col in desired_order if col in chelsa_columns]
    
    # Reorder columns
    ordered_columns = non_chelsa_columns + chelsa_columns
    return df[ordered_columns]

# Function to move 'lat' and 'lon' to specific positions
def move_lat_lon_columns(df, lat_pos=None, lon_pos=None):
    columns = df.columns.tolist()  # Get the list of columns
    if 'lat' in columns and lat_pos is not None:
        columns.insert(lat_pos, columns.pop(columns.index('lat')))  # Move 'lat' to the specified position
    if 'lon' in columns and lon_pos is not None:
        columns.insert(lon_pos, columns.pop(columns.index('lon')))  # Move 'lon' to the specified position
    return df[columns]  # Reorder DataFrame

# Load and rename columns for all DataFrames
current_df = pd.read_csv('data/precomputed/plot_and_abiotic_data_current.csv')
current_df = rename_chelsa_columns(current_df)

future_climate_ssp126 = pd.read_csv('data/precomputed/plot_and_abiotic_data_ssp126.csv')
future_climate_ssp126 = rename_chelsa_columns(future_climate_ssp126)

future_climate_ssp370 = pd.read_csv('data/precomputed/plot_and_abiotic_data_ssp370.csv')
future_climate_ssp370 = rename_chelsa_columns(future_climate_ssp370)

future_climate_ssp585 = pd.read_csv('data/precomputed/plot_and_abiotic_data_ssp585.csv')
future_climate_ssp585 = rename_chelsa_columns(future_climate_ssp585)
future_climate_ssp585 = rename_chelsa_columns(future_climate_ssp585)

# Determine the consistent CHELSA_bio column order
desired_order = get_consistent_chelsa_order(current_df, future_climate_ssp126, future_climate_ssp370, future_climate_ssp585)

# Apply the same column order to all DataFrames
current_df = reorder_columns(current_df, desired_order)
future_climate_ssp126 = reorder_columns(future_climate_ssp126, desired_order)
future_climate_ssp370 = reorder_columns(future_climate_ssp370, desired_order)
future_climate_ssp585 = reorder_columns(future_climate_ssp585, desired_order)

# Move 'lat' and 'lon' columns to specific positions
current_df = move_lat_lon_columns(current_df, lat_pos=26, lon_pos=27)
future_climate_ssp126 = move_lat_lon_columns(future_climate_ssp126, lat_pos=1, lon_pos=2)  
future_climate_ssp370 = move_lat_lon_columns(future_climate_ssp370, lat_pos=1, lon_pos=2)  
future_climate_ssp585 = move_lat_lon_columns(future_climate_ssp585, lat_pos=1, lon_pos=2)  

current_df

**Precipitation LOGOCV**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

# Load dataset
trait_names = current_df.columns[1:25].tolist()

# Define climate features
climate_features = [
    'EarthEnvTopoMed_Eastness', 'EarthEnvTopoMed_Elevation',
    'EarthEnvTopoMed_Northness', 'EarthEnvTopoMed_Slope',
    'SG_Bulk_density_015cm', 'SG_Clay_Content_015cm',
    'SG_Coarse_fragments_015cm', 'SG_Depth_to_bedrock',
    'SG_Sand_Content_015cm', 'SG_Silt_Content_015cm', 'CHELSA_bio1',
    'CHELSA_bio7', 'CHELSA_bio9', 'CHELSA_bio11', 'CHELSA_bio12',
    'CHELSA_bio15', 'CHELSA_bio17', 'CHELSA_bio19'
]

# Define precipitation buffer sizes
group_sizes_pr = np.arange(0, 160, 20)
bins_pr = np.arange(150, 6600, 20)

# Drop NaNs and reassign precipitation groups
current_df = current_df.dropna(subset=trait_names + climate_features)
current_df['precipitation_group_pr'] = pd.cut(current_df['CHELSA_bio12'], bins_pr, labels=bins_pr[:-1], include_lowest=True)
current_df = current_df.dropna(subset=['precipitation_group_pr'])
precipitation_labels_pr = current_df['precipitation_group_pr'].astype(np.int16).to_numpy()

# Extract features and targets
features = current_df[climate_features].values
targets = current_df.iloc[:, 1:len(trait_names) + 1].values

# Scale features and targets
scaler_features = StandardScaler()
scaler_targets = StandardScaler()
features_scaled = scaler_features.fit_transform(features)
targets_scaled = scaler_targets.fit_transform(targets)

assert features_scaled.shape[0] == targets_scaled.shape[0] == len(precipitation_labels_pr)

# Generate polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
features_poly = poly.fit_transform(features_scaled)

# Initialise Leave-One-Group-Out CV
logo = LeaveOneGroupOut()

# Store all predictions and true values
all_y_true = []
all_y_pred = []

# Perform LOGOCV
total_test_sets = 0
for group_size_pr in group_sizes_pr:
    print(f"\nCurrent group size: {group_size_pr} mm")
    
    for train_index, test_index in logo.split(features_poly, targets_scaled, groups=precipitation_labels_pr):
        test_group_index = precipitation_labels_pr[test_index[0]]
        if not (190 <= test_group_index <= 2010):
            continue
        
        exclude_groups = np.arange(test_group_index - group_size_pr, test_group_index + group_size_pr + 20, 20)
        train_mask = ~np.isin(precipitation_labels_pr, exclude_groups)
        
        features_train, features_test = features_poly[train_mask], features_poly[test_index]
        target_train, target_test = targets_scaled[train_mask], targets_scaled[test_index]
        
        print(f"Test group index: {test_group_index}, Exclude groups: [{exclude_groups}]")
        print(f"Number of training samples: {len(features_train)}, Number of test samples: {len(features_test)}")
        
        if len(features_train) == 0 or len(features_test) == 0:
            continue
        
        rf_regressor = RandomForestRegressor(n_estimators=1800, random_state=42, min_samples_split=2, 
                                             min_samples_leaf=1, max_features='log2', max_depth=20, n_jobs=-1, bootstrap=False)
        multioutput_rf = MultiOutputRegressor(rf_regressor)
        multioutput_rf.fit(features_train, target_train)
        
        rf_pred = multioutput_rf.predict(features_test)
        
        # Store results
        all_y_true.append(target_test)
        all_y_pred.append(rf_pred)
        
        # Compute per-test-set R squared and NRMSE
        r2_test = r2_score(target_test.ravel(), rf_pred.ravel())
        rmse_test = np.sqrt(mean_squared_error(target_test.ravel(), rf_pred.ravel()))
        nrmse_test = rmse_test / (np.max(target_test.ravel()) - np.min(target_test.ravel()))
        
        print(f"Test set R squared: {r2_test:.4f}, NRMSE: {nrmse_test:.4f}")
        total_test_sets += 1
    
# Concatenate all stored predictions and true values
all_y_true = np.vstack(all_y_true)
all_y_pred = np.vstack(all_y_pred)

# Compute overall R squared and NRMSE
final_r2 = r2_score(all_y_true.ravel(), all_y_pred.ravel())
final_rmse = np.sqrt(mean_squared_error(all_y_true.ravel(), all_y_pred.ravel()))
final_nrmse = final_rmse / (np.max(all_y_true.ravel()) - np.min(all_y_true.ravel()))

print(f"\nFinal Overall R squared: {final_r2:.4f}")
print(f"Final Overall NRMSE: {final_nrmse:.4f}")


In [ ]:
logocv_pr = pd.read_csv("data/precomputed/logocv_results_pr.csv")
logocv_pr

In [ ]:
# Define SSP precipitation ranges relative to the baseline
ssp_scenarios_pr = ['SSP126', 'SSP370', 'SSP585']
ssp_means_pr = [49.94, 64.05, 78.62]  # Mean pr increase relative to baseline
lower_confidence_pr = [3.59, 26.02, 4.37]  # Lower bound of 90% confidence interval. Absolute of negative values used
upper_confidence_pr = [97.5, 167.46, 185.79]  # Upper bound of 90% confidence interval. Absolute of negative values used

# Plotting overall R2 and NRMSE against degrees Celsius buffering
plt.figure(figsize=(10, 6), dpi=600)

# Plot R2 and NRMSE
plt.plot(logocv_pr.buffer_size, logocv_pr.R2, marker='o', linestyle='-', color='b', label='Weighted R-squared')
plt.plot(logocv_pr.buffer_size, logocv_pr.NRMSE, marker='s', linestyle='--', color='r', label='Weighted NRMSE')

# Define colors in a gradient from light green to light red
colors = ['lightgreen', 'orange', 'lightcoral']

# Plot SSP scenarios as shaded regions with labels and arrows
for ssp, mean, lower, upper, color in zip(ssp_scenarios_pr, ssp_means_pr, lower_confidence_pr, upper_confidence_pr, colors):
    plt.fill_betweenx([0, 1], lower, upper, color=color, alpha=0.25, edgecolor='black', linewidth=0.5, zorder=1)
    plt.annotate(ssp, 
                 xy=(mean, 0.9 - ssp_scenarios_pr.index(ssp) * 0.1), 
                 xytext=(0, -10), textcoords='offset points',
                 ha='center', va='top', fontsize=16, color='black', zorder=2)
    plt.annotate('', 
                 xy=(lower, 0.9 - ssp_scenarios_pr.index(ssp) * 0.1), 
                 xytext=(upper, 0.9 - ssp_scenarios_pr.index(ssp) * 0.1),
                 arrowprops=dict(arrowstyle='<->', color='black'), zorder=2)
    # Adding small vertical line for mean precipitation within the range
    plt.plot([mean, mean], [0.9 - ssp_scenarios_pr.index(ssp) * 0.1 - 0.02, 0.9 - ssp_scenarios_pr.index(ssp) * 0.1 + 0.02],
             color='black', linestyle='-', linewidth=0.5, zorder=3)

# Annotate each point with its value for R2 and NRMSE
for i, txt in enumerate(logocv_pr.R2):
    plt.annotate(f"{txt:.2f}", (logocv_pr.buffer_size[i], logocv_pr.R2[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', va='bottom', color='b', fontsize=16)

for i, txt in enumerate(logocv_pr.NRMSE):
    plt.annotate(f"{txt:.2f}", (logocv_pr.buffer_size[i], logocv_pr.NRMSE[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', color='r', fontsize=16)

# Set axes range and tick increments
plt.xticks(np.arange(0, max(logocv_pr.buffer_size) + 40, 40), fontsize=14)
plt.ylim(0, 1.0)
plt.yticks(np.arange(0, 1.1, 0.2), fontsize=16)

# Legend placement in the top right corner
plt.legend(loc='upper right', fontsize=14)

plt.xlabel('Precipitation (mm) buffering', fontsize=16)
plt.ylabel('Coefficient of determination (R2)', fontsize=16)
plt.title('', fontsize=16)
plt.grid(True)
plt.tight_layout()
plt.show()


**Temperature LOGOCV**

In [ ]:
# Temperature: Buffered LOGOCV using incremental 0.5± degrees celsius buffer. Metrics weighted by group size

# Load dataset
trait_names = current_df.columns[1:25].tolist()

# Define climate features
climate_features = [
    'EarthEnvTopoMed_Eastness', 'EarthEnvTopoMed_Elevation',
    'EarthEnvTopoMed_Northness', 'EarthEnvTopoMed_Slope',
    'SG_Bulk_density_015cm', 'SG_Clay_Content_015cm',
    'SG_Coarse_fragments_015cm', 'SG_Depth_to_bedrock',
    'SG_Sand_Content_015cm', 'SG_Silt_Content_015cm', 'CHELSA_bio1',
    'CHELSA_bio7', 'CHELSA_bio9', 'CHELSA_bio11', 'CHELSA_bio12',
    'CHELSA_bio15', 'CHELSA_bio17', 'CHELSA_bio19'
]

# Define bins and labels for temperature groups
group_sizes = np.arange(0, 4, 0.5)  # Group sizes in degrees Celsius to exclude
bins = np.arange(-9, 25, 0.5)  # Create bins from -7 to +10 with 0.5 degree intervals
labels = np.arange(-9, 24.5, 0.5)  # Create labels corresponding to the bins

# Drop NaNs and reassign temp groups
current_df = current_df.dropna(subset=trait_names + climate_features)
current_df['temperature_group'] = pd.cut(current_df['CHELSA_bio1'], bins, labels=bins[:-1], include_lowest=True)
current_df = current_df.dropna(subset=['temperature_group'])
temperature_labels = current_df['temperature_group'].astype(np.float32).to_numpy()

# Extract features and targets
features = current_df[climate_features].values
targets = current_df.iloc[:, 1:len(trait_names) + 1].values

# Scale features and targets
scaler_features = StandardScaler()
scaler_targets = StandardScaler()
features_scaled = scaler_features.fit_transform(features)
targets_scaled = scaler_targets.fit_transform(targets)

assert features_scaled.shape[0] == targets_scaled.shape[0] == len(temperature_labels)

# Generate polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
features_poly = poly.fit_transform(features_scaled)

# Initialise Leave-One-Group-Out CV
logo = LeaveOneGroupOut()

# Store all predictions and true values
all_y_true = []
all_y_pred = []

# Perform LOGOCV
total_test_sets = 0
for group_size in group_sizes:
    print(f"\nCurrent group size: {group_size} C")
    
    for train_index, test_index in logo.split(features_poly, targets_scaled, groups=temperature_labels):
        test_group_index = temperature_labels[test_index[0]]
            
        exclude_groups = np.arange(test_group_index - group_size, test_group_index + group_size + 0.5, 0.5)
        train_mask = ~np.isin(temperature_labels, exclude_groups)
        
        features_train, features_test = features_poly[train_mask], features_poly[test_index]
        target_train, target_test = targets_scaled[train_mask], targets_scaled[test_index]
        
        print(f"Test group index: {test_group_index}, Exclude groups: [{exclude_groups}]")
        print(f"Number of training samples: {len(features_train)}, Number of test samples: {len(features_test)}")
        
        if len(features_train) == 0 or len(features_test) == 0:
            continue
        
        rf_regressor = RandomForestRegressor(n_estimators=1800, random_state=42, min_samples_split=2, 
                                             min_samples_leaf=1, max_features='log2', max_depth=20, n_jobs=-1, bootstrap=False)
        multioutput_rf = MultiOutputRegressor(rf_regressor)
        multioutput_rf.fit(features_train, target_train)
        
        rf_pred = multioutput_rf.predict(features_test)
        
        # Store results
        all_y_true.append(target_test)
        all_y_pred.append(rf_pred)
        
        # Compute per-test-set R squared and NRMSE
        r2_test = r2_score(target_test.ravel(), rf_pred.ravel())
        rmse_test = np.sqrt(mean_squared_error(target_test.ravel(), rf_pred.ravel()))
        nrmse_test = rmse_test / (np.max(target_test.ravel()) - np.min(target_test.ravel()))
        
        print(f"Test set R squared: {r2_test:.4f}, NRMSE: {nrmse_test:.4f}")
        total_test_sets += 1
    
# Concatenate all stored predictions and true values
all_y_true = np.vstack(all_y_true)
all_y_pred = np.vstack(all_y_pred)

# Compute overall R squared and NRMSE
final_r2 = r2_score(all_y_true.ravel(), all_y_pred.ravel())
final_rmse = np.sqrt(mean_squared_error(all_y_true.ravel(), all_y_pred.ravel()))
final_nrmse = final_rmse / (np.max(all_y_true.ravel()) - np.min(all_y_true.ravel()))

print(f"\nFinal Overall R squared: {final_r2:.4f}")
print(f"Final Overall NRMSE: {final_nrmse:.4f}")

In [ ]:
logocv_temp = pd.read_csv("data/logocv_results_temp.csv")
logocv_temp

In [ ]:
# Define SSP temperature ranges relative to the baseline
ssp_scenarios = ['SSP126', 'SSP370', 'SSP585']
ssp_means = [1.59, 4.1, 5.47]  # Mean temperature increase relative to baseline
lower_confidence = [1.0, 3.09, 4.01]  # Lower bound of 90% confidence interval
upper_confidence = [2.63, 5.72, 7.29]  # Upper bound of 90% confidence interval

import matplotlib.pyplot as plt
import numpy as np

# Plotting overall R2 and NRMSE against degrees Celsius buffering
plt.figure(figsize=(10, 6), dpi=600)

# Plot R2 and NRMSE
plt.plot(logocv_temp.buffer_size, logocv_temp.R2, marker='o', linestyle='-', color='b', label='Weighted R-squared')
plt.plot(logocv_temp.buffer_size, logocv_temp.NRMSE, marker='s', linestyle='--', color='r', label='Weighted NRMSE')

# Define colors in a gradient from light green to light red
colors = ['lightgreen', 'orange', 'lightcoral']

# Plot SSP scenarios as shaded regions with labels and arrows
for ssp, mean, lower, upper, color in zip(ssp_scenarios, ssp_means, lower_confidence, upper_confidence, colors):
    plt.fill_betweenx([0, 1], lower, upper, color=color, alpha=0.25, edgecolor='black', linewidth=0.5, zorder=1)
    plt.annotate(
        ssp, 
        xy=(mean, 0.9 - ssp_scenarios.index(ssp) * 0.1), 
        xytext=(0, -10), 
        textcoords='offset points',
        ha='center', 
        va='top', 
        fontsize=16, 
        color='black', 
        zorder=2
    )
    plt.annotate(
        '', 
        xy=(lower, 0.9 - ssp_scenarios.index(ssp) * 0.1), 
        xytext=(upper, 0.9 - ssp_scenarios.index(ssp) * 0.1),
        arrowprops=dict(arrowstyle='<->', color='black'),
        zorder=2
    )
    # Adding a small vertical line for the mean temperature within the range
    plt.plot(
        [mean, mean], 
        [0.9 - ssp_scenarios.index(ssp) * 0.1 - 0.02, 0.9 - ssp_scenarios.index(ssp) * 0.1 + 0.02],
        color='black', 
        linestyle='-', 
        linewidth=0.5, 
        zorder=3
    )

# Annotate each point with its value for R2 and NRMSE
for i, txt in enumerate(logocv_temp.R2):
    plt.annotate(
        f"{txt:.2f}", 
        (logocv_temp.buffer_size[i], logocv_temp.R2[i]), 
        textcoords="offset points", 
        xytext=(0, 10), 
        ha='center', 
        va='bottom', 
        color='b', 
        fontsize=16
    )

for i, txt in enumerate(logocv_temp.NRMSE):
    plt.annotate(
        f"{txt:.2f}", 
        (logocv_temp.buffer_size[i], logocv_temp.NRMSE[i]), 
        textcoords="offset points", 
        xytext=(0, 10), 
        ha='center', 
        color='r', 
        fontsize=16
    )

# Set axes range and tick increments with larger fonts
plt.xticks(np.arange(0, max(logocv_temp.buffer_size) + 1, 1), fontsize=14)
plt.ylim(0, 1.0)
plt.yticks(np.arange(0, 1.1, 0.2), fontsize=16)

# Legend placement in the top right corner with increased font size
plt.legend(loc='upper right', fontsize=14)

plt.xlabel('Degrees Celsius buffering', fontsize=16)
plt.ylabel('Coefficient of determination (R2)', fontsize=16)
plt.title('', fontsize=16)
plt.grid(True)
plt.tight_layout()
plt.show()

